In [1]:
'''
    Created on April 12th 2017
    Updated on April 20th 2019 
    
    A simple and didactic one hidden Multilayer Perceptron
    
    @author: Altino Dantas
'''
import copy
import math
import numpy as np
from random import random

def PMC(inputs, outputs, W1NumberOfNeurons, W2NumberOfNeurons, alpha, epsilon, max_it, loss_path='dados.dat'):
    
    '''
       ############# Initialize vars #############
    '''
    
    times = 1
    
    W2, W2Aux, W1 = [],[],[]

    # neuron j weight i, thus, they will seen W = [[0.35, 0.15, 0.2], [0.35, 0.25, 0.3]]
    
    # set random W1 weights // bias is the first weight in each neuron;
    for neuron in range(W1NumberOfNeurons):
        W1.append([])
        # the number of weights for each neuron in W1 is the number of features in the sample
        for weight in range(len(inputs[0])):
            W1[neuron].append(random())

    # set random W2 weights // bias is the first weight in each neuron; 
    for neuron in range(W2NumberOfNeurons):
        W2.append([])
        # num of weights linked with each W2 neuron is the num of outs from Layer 1 plus one (regarding to bias)
        for weight in range(len(W1) + 1): 
            W2[neuron].append(random())       

    # activation power u
    U1 = [0 for x in range(W1NumberOfNeurons)]
    U2 = [0 for x in range(W2NumberOfNeurons)]

    # output produced by g(u)
    Y1 = [0 for x in range(W1NumberOfNeurons)] 
    Y2 = [0 for x in range(W2NumberOfNeurons)]
    
    # Instrumentation to save loss in a external file
    outTXT = open(loss_path, 'w')
    textLines = []

    '''
       ############# Training loop #############
    '''
    
    while True:

        iniErro = ETotal(W1,W2,U1,U2,Y1,Y2,inputs,outputs)

        '''
            ############# iterate over each sample #############
        '''
        
        for i in range(len(inputs)):
            
            '''
                ############# propagation step #############
            '''
            
            U1 = updateU1(inputs[i],U1, W1)
            Y1 = updateY1(U1,Y1)
            U2 = updateU2(Y1,U2,W2)
            Y2 = updateY2(U2, Y2)       

            '''
                ############# backpropagation step #############
            '''
            W2Aux = copy.deepcopy(W2)
            
            # updating first level (last layer)
            for neuron in range(len(W2)):
                W2[neuron][0] += 1 * alpha * (outputs[i][neuron] - Y2[neuron]) * (Y2[neuron] * (1 - Y2[neuron])) #bias    
                for weight in range(1, len(W2[neuron])):
                    W2[neuron][weight] += Y1[weight-1] * alpha * \
                    (outputs[i][neuron] - Y2[neuron]) * (Y2[neuron] * (1 - Y2[neuron]))

            # updating second level (first hidden layer)
            for neuron in range(len(W1)):
                for weight in range(len(W1[neuron])):
                    js = .0
                    for neuronW2 in range(len(W2Aux)):
                        js += W2Aux[neuronW2][neuron + 1] * \
                        (outputs[i][neuronW2] - Y2[neuronW2]) * (Y2[neuronW2] * (1 - Y2[neuronW2]))
                    W1[neuron][weight] += inputs[i][weight] * alpha * js * (Y1[neuron] * (1 - Y1[neuron])) 

        endErro = ETotal(W1,W2,U1,U2,Y1,Y2,inputs,outputs)

        if (math.fabs(iniErro - endErro) < epsilon) or (times == max_it):
            break 
        else:
            if times % 100 == 0:
                print('{:d}'.format(times))
            
            string = str(iniErro) + "\n"
            textLines.append(string)
            times += 1
    
    # save loss in file
    outTXT.writelines(textLines)
    outTXT.close()
    
    return (W1,W2)

def updateU1(anInput,U1,W1): 
    for neuron in range(len(W1)):
        U1[neuron] = 0
        for k in range(len(anInput)):
            U1[neuron]  += anInput[k] * W1[neuron][k]  
    return U1

def updateY1(uVec, Y1): 
    for neuron in range(len(Y1)):
        Y1[neuron] = funcao_logistica(uVec[neuron])
    return Y1

def updateU2(anInput,U2, W2): 
    for neuron in range(len(U2)):
        U2[neuron] = W2[neuron][0] # initializing each neuron u value with bias from W2 weight 
        for weight in range(len(anInput)):
            U2[neuron] += anInput[weight] * W2[neuron][weight + 1]
    return U2
            
def updateY2(uVec,Y2): 
    for i in range(len(uVec)):
        Y2[i]  = funcao_logistica(uVec[i])
    return Y2
        
def funcao_logistica(u):
    return 1/(1 + np.exp(-u));

def ETotal(W1,W2,U1,U2,Y1,Y2,inputs,outputs):
    average = .0    
    for j in range( len(inputs) ):
        U1 = updateU1(inputs[j],U1,W1)
        Y1 = updateY1(U1,Y1)
        U2 = updateU2(Y1, U2,W2)
        Y2 = updateY2(U2,Y2)
        for i in range( len( W2 ) ): # for all neurons
            average += (math.pow(Y2[i] - outputs[j][i], 2))/len(W2)
            
    return ( average / len( inputs ) )

def predict(sample, W1, W2):
    
    # activation power u
    U1 = [0 for x in range(len(W1))]
    U2 = [0 for x in range(len(W2))]

    # output produced by g(u)
    Y1 = [0 for x in range(len(W1))] 
    Y2 = [0 for x in range(len(W2))]
    
    '''
        ############# propagation step #############
    '''
    
    U1 = updateU1(sample,U1, W1)
    Y1 = updateY1(U1,Y1)
    U2 = updateU2(Y1,U2,W2)
    Y2 = updateY2(U2, Y2)
    
    return Y2

        
def main():
    
    W1NumberOfNeurons = 2     
    W2NumberOfNeurons = 1 

    alpha   = 0.1
    epsilon = .0000001
    max_it  = 1000 

    # first value in each input is the constant input for bias
    inputs = [[1,0,0],[1,0,1],[1,1,0],[1,1,1]]
    outputs = [[0],[1],[1],[0]]

    W1, W2 = PMC(inputs=inputs, 
                 outputs=outputs, 
                 W1NumberOfNeurons=W1NumberOfNeurons, 
                 W2NumberOfNeurons=W2NumberOfNeurons, 
                 alpha=alpha,
                 epsilon=epsilon,
                 max_it=max_it)
    
    tests =[[1,0,0],
            [1,0,1],
            [1,1,0],
            [1,1,1]]
    
    for t in tests:
        print(predict(t,W1,W2))

if __name__ == '__main__':
    main()

100
200
300
400
500
600
700
800
900
[0.50487227332410989]
[0.5040141081036229]
[0.49622826691646965]
[0.49552090094447721]
